In [1]:
import pandas as pd
import numpy as np
from collections import Counter 
#from Bio.Seq import Seq 
import itertools as itr 
import distance 
import datetime
from itertools import combinations

debemos buscar aquella cadena k=10 posiciones que más se repite dentro de dataset con d=2 mismatches de la siguiente cadena: 

In [3]:
dataset='CACAGTAGGCGCCGGCACACACAGCCCCGGGCCCCGGGCCGCCCCGGGCCGGCGGCCGCCGGCGCCGGCACACCGGCACAGCCGTACCGGCACAGTAGTACCGGCCGGCCGGCACACCGGCACACCGGGTACACACCGGGGCGCACACACAGGCGGGCGCCGGGCCCCGGGCCGTACCGGGCCGCCGGCGGCCCACAGGCGCCGGCACAGTACCGGCACACACAGTAGCCCACACACAGGCGGGCGGTAGCCGGCGCACACACACACAGTAGGCGCACAGCCGCCCACACACACCGGCCGGCCGGCACAGGCGGGCGGGCGCACACACACCGGCACAGTAGTAGGCGGCCGGCGCACAGCC'


Generamos todas las cadenas de tamaño k=10 consecutivas que existen dentro de la secuencia:

In [3]:
kmers = [dataset[i:i+10] for i in range(len(dataset)-9)]
kmers[0:5]

['CACAGTAGGC', 'ACAGTAGGCG', 'CAGTAGGCGC', 'AGTAGGCGCC', 'GTAGGCGCCG']

In [4]:
len(kmers)

352

existen 352 secuencias de k=10 posiciones dentro de dataset

# Metodología

Paso1.- vamos a generar un diccionario con las cadenas de k=10 posiciones únicas con su respectiva frecuencia de apariciones dentro de dataset.

In [5]:
initcount  = Counter(kmers) #contamos el número de veces que aparece cada secuencia de k=10 en dataset
kmer2count = dict(initcount) #tenemos un dictionario de cadenas con sus frecuencias como valores

obtenemos un diccionario:

{'ACACACACAC': 1,
 'ACACACACAG': 1,
 'ACACACACCG': 2, ....

Paso 2.-para cada $$ s \in dataset$$ vamos a generar un conjunto de secuencias $$s'=(s'_{1},...,s'_{n})$$ tales que $$ d(s,s'_{i})<=2 \ \forall s'_{i} \in s' \ and \ s'_{i}\ne s $$  se genera dicha función:

In [6]:
from itertools import combinations, product

def generate(s, d):
    N = len(s)
    letters = 'ACGT'
    pool = list(s)

    for indices in combinations(range(N), d):
        for replacements in product(letters, repeat=d):
            skip = False
            for i, a in zip(indices, replacements):
                if pool[i] == a: skip = True
            if skip: continue

            keys = dict(zip(indices, replacements))
            yield ''.join([pool[i] if i not in indices else keys[i] 
                           for i in range(N)])

Paso 3.- al momento de tener $$s'=(s'_{1},...,s'_{n})$$ vamos a revisar si $$ s'_{i} \in kmer2count?$$ es decir si la secuencia generada está o no dentro del diccionario y entonces tenemos que $$ si \ [s'_{i }:n] \in kmer2count \ entonces \ [s'_{i}:n+1]$$ sumamos 1 al valor de la cadena en el diccionario si no está en kmer2count entonces lo agregamos al diccionario kmer2count con valor 1 $$[s'_{i}:1]$$

paso2 y 3 se hace en cada iteración es decir mientras se va recorriendo la secuencia dataset de 10 en 10 de tal modo que al finalizar el recorrido ya tenemos los conteos finales para todas las cadenas de tamaño 10 con mismatch 2.

In [7]:
now=datetime.datetime.now()


for k in range(len(kmers)):
    aux=list(generate(list(kmers)[k],2))+list(generate(list(kmers)[k],1)) #pso 2 para cada s en kmers
    #generate(list(kmers)[k],2) genera todas las cadenas con un mismatch exacto de 2 por ello agregamos 
    #los mismatch de 1 usando generate(list(kmers)[k],1)
    
    for w in range(len(aux)):
        
        if aux[w] in kmer2count:
            kmer2count[aux[w]]+=1    #paso 3 para toda s' generada por s 
        else:
            kmer2count[aux[w]] = 1
#    print(k,len(list(kmer2count)),len(aux),a in aux,a in kmer2count,len(Counter(list(kmer2count))))
z=list(kmer2count)
q=np.array(list(kmer2count.values()))
v=np.where(q==max(q))[0]
for n in range(len(v)):
    print(z[v[n]]," ",q[v[n]],"pertenece a la secuencia:",z[v[n]] in kmers)

fin=datetime.datetime.now()
print("tiempo total: ",fin-now) 

GCGCACACAC   20 pertenece a la secuencia: True
GCACACAGAC   20 pertenece a la secuencia: False
tiempo total:  0:00:01.080404


la salida imprime la(s) cadena(s) más frecuentes y además indica si esta(s) está(n) o no en la secuencia original. 
tiempo total: 1 seg
respuestas:
GCGCACACAC 
GCACACAGAC
cadenas más frecuentes.

In [8]:
def kmers_mm(s,k,d):
    kmers = [s[i:i+k] for i in range(len(s)+k-1)] 
    initcount  = Counter(kmers) #contamos el número de veces que aparece cada secuencia de k=10 en dataset
    kmer2count = dict(initcount) #tenemos un dictionario de cadenas con sus frecuencias como valores
    for k in range(len(kmers)):
        aux=list(generate(list(kmers)[k],2))+list(generate(list(kmers)[k],1)) #pso 2 para cada s en kmers
        #generate(list(kmers)[k],2) genera todas las cadenas con un mismatch exacto de 2 por ello agregamos 
        #los mismatch de 1 usando generate(list(kmers)[k],1)        
    
        for w in range(len(aux)):
        
            if aux[w] in kmer2count: #si está en el diccionario suma 1
                kmer2count[aux[w]]+=1    #paso 3 para toda s' generada por s 
            else:
                kmer2count[aux[w]] = 1 #si o está agrégalo
#    print(k,len(list(kmer2count)),len(aux),a in aux,a in kmer2count,len(Counter(list(kmer2count))))
    z=list(kmer2count)
    q=np.array(list(kmer2count.values()))
    v=np.where(q==max(q))[0] #genera los índices de las secuencias con mayor frecuencia
    for n in range(len(v)):
        print(z[v[n]]," ",q[v[n]],"pertenece a la secuencia:",z[v[n]] in kmers) #resultado


In [9]:
#probamos la función completa
now=datetime.datetime.now() 
kmers_mm(dataset,10,2)

fin=datetime.datetime.now()
print("tiempo total: ",fin-now) 

GCGCACACAC   20 pertenece a la secuencia: True
GCACACAGAC   20 pertenece a la secuencia: False
tiempo total:  0:00:01.063843
